# 🔍 YOLOv8 + Roboflow + hyp.yaml ile Eğitim

Bu notebook, Roboflow'dan veri indirerek YOLOv8 ile özel veri setinizde eğitim yapar. Ayrıca özel hiperparametre ayarlarını `hyp.yaml` dosyası üzerinden uygular.

In [1]:
# 🔧 Gerekli kütüphaneleri kur
!pip install -q roboflow ultralytics


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.1.1 which is incompatible.


In [2]:
# 📦 Roboflow'dan veri çekme
from roboflow import Roboflow

# Roboflow API anahtarınızı buraya yazın
rf = Roboflow(api_key="ROBOFLOW_API_KEYINIZI_YAZIN")
project = rf.workspace("WORKSPACE_ADI").project("PROJE_ADI")  # örnek: rf.workspace("enes-works").project("iha-tespiti")
dataset = project.version(1).download("yolov8")

# Dataset yolu
data_yaml_path = f"{dataset.location}/data.yaml"


upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=undefined&ref=undefined
loading Roboflow workspace...


RoboflowError: {
    "error": {
        "message": "Unsupported get request. Workspace with ID \"WORKSPACE_ADI\" does not exist or cannot be loaded due to missing permissions.",
        "status": 404,
        "type": "GraphMethodException",
        "hint": "You can see your available workspaces by issuing a GET request to /workspaces"
    }
}

: 

In [ ]:
# ✏️ hyp.yaml dosyasını oluştur
hyp_yaml = """lr0: 0.01
momentum: 0.937
weight_decay: 0.0005
hsv_h: 0.015
hsv_s: 0.7
hsv_v: 0.4
degrees: 0.0
translate: 0.1
scale: 0.5
shear: 0.0
flipud: 0.0
fliplr: 0.5
mosaic: 1.0
mixup: 0.0
"""

with open("hyp.yaml", "w") as f:
    f.write(hyp_yaml)


In [ ]:
# 🧠 YOLOv8 modelini yükle ve eğitimi başlat
from ultralytics import YOLO

model = YOLO("yolov8s.pt")  # Küçük model, iyi bir başlangıçtır

model.train(
    data=data_yaml_path,
    epochs=100,
    imgsz=640,
    batch=16,
    name="roboflow_custom_model",
    hyp="hyp.yaml"
)


In [ ]:
# 🔍 Test için tahmin
model = YOLO("runs/detect/roboflow_custom_model/weights/best.pt")

# Örnek görsel ile tahmin (test klasöründen bir görsel yolu girin)
results = model.predict(source=f"{dataset.location}/test/images", save=True)
